In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"jawadbenali","key":"bfa9f810facbf1bd72e164820828d7f9"}'}

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c dogs-vs-cats

100% 812M/812M [00:08<00:00, 118MB/s] 
100% 812M/812M [00:08<00:00, 105MB/s]


In [7]:
!unzip -qq dogs-vs-cats.zip

In [8]:
!unzip -qq train.zip

In [9]:
import pandas as pd
import os, shutil, pathlib
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers
from IPython.display import Image

In [10]:
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")
def make_subset(subset_name, start_index, end_index):
  for category in ("cat", "dog"):
    dir = new_base_dir / subset_name / category
    os.makedirs(dir)
    fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
    for fname in fnames:
        shutil.copyfile(src=original_dir / fname, dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [11]:
inputs = keras.Input(shape=(180,180,3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters = 32,kernel_size = 3,activation = "relu")(x)
x = layers.MaxPooling2D(pool_size =2)(x)
x = layers.Conv2D(filters = 64,kernel_size = 3,activation = "relu")(x)
x = layers.MaxPooling2D(pool_size =2)(x)
x = layers.Conv2D(filters = 128,kernel_size = 3,activation = "relu")(x)
x = layers.MaxPooling2D(pool_size =2)(x)
x = layers.Conv2D(filters = 256,kernel_size = 3,activation = "relu")(x)
x = layers.MaxPooling2D(pool_size =2)(x)
x = layers.Conv2D(filters = 256,kernel_size = 3,activation = "relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 89, 89, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 43, 43, 64)        0         
 g2D)                                                        

In [13]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [14]:
train_ds = image_dataset_from_directory(
    new_base_dir / "train",
    image_size = (180,180),
    batch_size = 32,
    )
validation_ds = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size = (180,180),
    batch_size = 32,
    )
test_ds = image_dataset_from_directory(
    new_base_dir / "test",
    image_size = (180,180),
    batch_size = 32,
    )

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [15]:
for data_batch, labels_batch in train_ds:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

data batch shape: (32, 180, 180, 3)
labels batch shape: (32,)


In [16]:
history = model.fit(
train_ds,
epochs=10,
validation_data=validation_ds)

Epoch 1/10
63/63 [==============================] - 12s 81ms/step - loss: 0.7111 - accuracy: 0.5225 - val_loss: 0.7186 - val_accuracy: 0.5000
Epoch 2/10
63/63 [==============================] - 4s 58ms/step - loss: 0.6938 - accuracy: 0.5405 - val_loss: 0.6874 - val_accuracy: 0.5130
Epoch 3/10
63/63 [==============================] - 7s 103ms/step - loss: 0.6845 - accuracy: 0.5700 - val_loss: 0.6675 - val_accuracy: 0.5920
Epoch 4/10
63/63 [==============================] - 4s 56ms/step - loss: 0.6638 - accuracy: 0.6175 - val_loss: 0.6501 - val_accuracy: 0.6170
Epoch 5/10
63/63 [==============================] - 4s 57ms/step - loss: 0.6530 - accuracy: 0.6430 - val_loss: 0.6869 - val_accuracy: 0.5870
Epoch 6/10
63/63 [==============================] - 5s 81ms/step - loss: 0.6161 - accuracy: 0.6730 - val_loss: 0.6048 - val_accuracy: 0.6810
Epoch 7/10
63/63 [==============================] - 4s 59ms/step - loss: 0.5804 - accuracy: 0.6985 - val_loss: 0.5772 - val_accuracy: 0.6900
Epoch 8/10


In [17]:
import cv2
image_path = 'cats_vs_dogs_small/train/cat/cat.2.jpg'
image = cv2.imread(image_path)
print(image.shape)

(396, 312, 3)
